<a href="https://colab.research.google.com/github/Geckomonc/AI20242-GeraldineAcevedo-MairaRodas/blob/main/04%20-%20preprocesado%20original%20%26%20CatBoostClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Kaggle

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


#Descargando la data de Kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 60% 12.0M/20.1M [00:00<00:00, 30.7MB/s]
100% 20.1M/20.1M [00:00<00:00, 50.9MB/s]


#Unzip and inspect data

In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


#Cargar datos de entrenamiento con panda

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv("train.csv")
data

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-alto
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,No,bajo
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-bajo
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,Primaria completa,Si,bajo


In [ ]:
submission_example = pd.read_csv('submission_example.csv')
submission_example

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo
...,...,...
296781,496981,alto
296782,209415,medio-alto
296783,239074,medio-alto
296784,963852,medio-bajo


In [ ]:
t = pd.read_csv('test.csv')
t

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
...,...,...,...,...,...,...,...,...,...,...,...,...
296781,296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Primaria incompleta,Si
296782,296782,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Educación profesional completa,No
296783,296783,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Educación profesional completa,No
296784,296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Educación profesional completa,No


#Limpieza de datos realizada en la segunda entrega

In [ ]:
#Agrupación de datos por programa académico
grupo_dict = {
    'CIENCIAS_SOCIALES': [
        'DERECHO','LICENCIATURA EN CIENCIAS SOCIALES', 'LICENCIATURA EN ETNOEDUCACION','CIENCIAS POLITICAS',
        'LICENCIATURA EN FILOSOFIA', 'LICENCIATURA EN CIENCIAS RELIGIOSAS','CIENCIA POLITICA',
        'TRABAJO SOCIAL', 'LICENCIATURA EN FILOSOFÍA', 'RELACIONES INTERNACIONALES', 'JURISPRUDENCIA',
        'SOCIOLOGIA', 'FILOSOFIA', 'LICENCIATURA EN FILOSOFIA Y EDUCACION RELIGIOSA', 'LICENCIATURA EN TEOLOGIA',
        'PLANEACION Y DESARROLLO SOCIAL', 'PROFESIONAL EN CRIMINALISTICA',
        'LICENCIATURA EN ETNOEDUCACION CON ENFASIS EN CIENCIAS SOCIALES', 'CIENCIA POLÍTICA Y GOBIERNO',
        'ANTROPOLOGIA', 'CIENCIAS BIBLICAS', 'HISTORIA',  'PROFESIONAL EN TRABAJO SOCIAL', 'CIENCIA POLÍTICA',
        'TEOLOGIA', 'CIENCIA POLITICA Y GOBIERNO', 'CIENCIA POLITICA Y RELACIONES INTERNACIONALES',
        'RELACIONES INTERNACIONALES Y ESTUDIOS POLITICOS',  'GOBIERNO Y RELACIONES INTERNACIONALES',
        'GESTION CULTURAL Y COMUNICATIVA', 'LICENCIATURA EN ETNOEDUCACION Y DESARROLLO COMUNITARIO',
        'DERECHO Y CIENCIAS POLITICAS', 'CRIMINALISTICA', 'LICENCIATURA EN CIENCIAS RELIGIOSAS',
        'CIENCIAS POLÍTICAS', 'TEOLOGÍA', 'FILOSOFIA Y LETRAS', 'FILOSOFÍA', 'HISTORIA Y PATRIMONIO',
        'GERENCIA DE LA SEGURIDAD Y ANALISIS SOCIO POLITICO', 'LICENCIATURA EN FILOSOFIA',
        'INVESTIGACIÓN CRIMINAL', 'LICENCIATURA EN TEOLOGÍA',
        'DESARROLLO FAMILIAR','LICENCIATURA EN HISTORIA','SOCIOLOGÍA','POLITICA Y RELACIONES INTERNACIONALES',
        'DERECHO Y CIENCIAS SOCIALES', 'GOBIERNO Y ASUNTOS PUBLICOS','ESTUDIOS EN FILOSOFIA',
        'LICENCIATURA EN CIENCIAS SOCIALES Y DESARROLLO LOCAL',
        'PERIODISMO Y OPINION PUBLICA','FILOSOFÍA Y HUMANIDADES','DESARROLLO TERRITORIAL',
        'GEOGRAFIA DEL DESARROLLO REGIONAL Y AMBIENTAL','DERECHO Y CIENCIAS POLÍTICAS',
        'ACUICULTURA','LICENCIATURA EN EDUCACION COMUNITARIA CON ENFASIS EN DERECHOS HUMANOS',
        'CIENCIA POLITICA, GOBIERNO Y RELACIONES INTERNACIONALES','ESTUDIOS Y GESTIÓN CULTURAL',
        'FILOSOFIA Y HUMANIDADES', 'TERAPIAS PSICOSOCIALES', 'ANTROPOLOGÍA',
        'PROFESIONAL EN FILOSOFIA', 'LENGUAJES Y ESTUDIOS SOCIOCULTURALES', 'URBANISMO',
        'ESTUDIOS Y GESTION CULTURAL','ESTUDIOS POLITICOS Y RESOLUCION DE CONFLICTOS',
        'LICENCIATURA EN EDUCACION BASICA ENFASIS EN CIENCIAS SOCIALES',
        'GESTIÓN CULTURAL','GESTIÓN  DE SISTEMAS DE INFORMACIÓN','INVESTIGACION CRIMINAL',
        'PROFESIONAL EN INVESTIGACIÓN CRIMINAL', 'ARQUEOLOGIA',
        ],

    'CIENCIAS_ECONOMICAS': [
        'ECONOMIA', 'MERCADEO Y PUBLICIDAD', 'CONTADURIA PÚBLICA', 'CONTADURÍA PÚBLICA', 'MARKETING Y LOGISTICA',
        'CONTADURIA PUBLICA', 'NEGOCIOS INTERNACIONALES', 'FINANZAS Y COMERCIO INTERNACIONAL',
        'PROFESIONAL EN FINANZAS Y NEGOCIOS INTERNACIONALES', 'FINANZAS Y NEGOCIOS INTERNACIONALES',
        'ECONOMÍA','MERCADEO Y NEGOCIOS INTERNACIONALES', 'ECONOMIA Y FINANZAS INTERNACIONALES',
        'MERCADEO', 'ECONOMIA Y NEGOCIOS INTERNACIONALES', 'NEGOCIOS Y RELACIONES INTERNACIONALES',
        'NEGOCIOS INTERNACIONALES SEGUNDO CICLO',  'PROFESIONAL EN CONTADURIA PUBLICA', 'PROFESIONAL EN AGRONEGOCIOS',
        'RELACIONES ECONOMICAS INTERNACIONALES', 'ADMINISTRACION BANCARIA Y FINANCIERA', 'MERCADOLOGIA',
        'COMERCIO Y NEGOCIOS INTERNACIONALES', 'FINANZAS Y COMERCIO EXTERIOR', 'COMERCIO EXTERIOR',
        'NEGOCIOS Y FINANZAS INTERNACIONALES', 'COMERCIO INTERNACIONAL Y FINANZAS', 'FINANZAS Y RELACIONES INTERNACIONALES',
        'COMERCIO INTERNACIONAL', 'FINANZAS', 'CONTADURIA PUBLICA Y FINANZAS INTERNACIONALES',
        'PROFESIONAL EN BANCA Y FINANZAS', 'CONTADURÍA PÚBLICA.', 'CONTADURIA', 'PROFESIONAL EN NEGOCIOS INTERNACIONALES',
        'PROFESIONAL EN MARKETING Y NEGOCIOS INTERNACIONALES', 'MERCADEO NACIONAL E INTERNACIONAL',
        '3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES', 'MARKETING Y NEGOCIOS INTERNACIONALES',
        'PRODUCCION AGROINDUSTRIAL','CONTADURÍA INTERNACIONAL','PUBLICIDAD INTERNACIONAL','PROFESIONAL EN MERCADEO',
        'CONTADURÍA PUBLICA','BANCA Y FINANZAS','PROFESIONAL EN MERCADEO EMPRESARIAL',
        'MERCADEO AGROINDUSTRIAL','ECONOM¿A','MERCADEO INTERNACIONAL Y PUBLICIDAD',
        'CONTADURIA P¿BLICA','LOGISTICA Y MERCADEO ORGANIZACIONAL',
        'CONTADURIA PUBLICA ENFASIS EN SISTEMAS Y ECONOMIA SOLIDARIA', 'MERCADEO Y VENTAS',
        'PROFESIONAL EN MARKETING INTERNACIONAL','GESTION EN ECOLOGIA Y TURISMO',
        'COMERCIO Y FINANZAS INTERNACIONALES', 'PROFESIONAL EN PUBLICIDAD Y MERCADEO',
        'BANCA Y FINANZAS INTERNACIONALES',
        ],

    'PEDAGOGIA': [
        'LICENCIATURA EN PEDAGOGIA INFANTIL', 'LICENCIATURA EN EDUCACION PREESCOLAR',
        'LICENCIATURA EN PSICOLOGIA Y PEDAGOGIA', 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN EDUCACION BÁSICA CON ÉNFASIS EN CIENCIAS NATURALES Y EDUCACIÓN AMBIENTAL',
        'LICENCIATURA EN EDUCACION PREESCOLAR', 'LICENCIATURA EN PSICOLOGIA Y PEDAGOGIA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES E IDIOMAS', 'LICENCIATURA EN PEDAGOGÍA INFANTIL',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTES',
        'LICENCIATURA EN EDUCACION PARA LA PRIMERA INFANCIA', 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICAS',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN INGLÉS', 'LICENCIATURA EN EDUCACIÓN PARA LA PRIMERA INFANCIA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN LENGUA CASTELLANA', 'LICENCIATURA EN EDUCACION INFANTIL',
        'LICENCIATURA EN EDUCACIÓN BÁSICA PRIMARIA', 'LICENCIATURA EN EDUCACIÓN INFANTIL',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICA HUMANIDADES Y LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTES',
        'LICENCIATURA EN PEDAGOGIA  INFANTIL', 'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN EDUCACIÓN ARTÍSTICA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN LENGUAS EXTRANJERAS', 'LICENCIATURA EN EDUCACION ESPECIAL',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E INGLES',
        'LICENCIATURA EN EDUCACION CON ENFASIS EN EDUCACION ESPECIAL', 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN INGLES',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS SOCIALES', 'LICENCIATURA EN EDUCACION POPULAR',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES -  LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTE',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES- LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES, LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACIÓN BÁSICA, CON ÉNFASIS EN HUMANIDADES, LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN CIENCIAS SOCIALES', 'LICENCIATURA EN EDUCACION AMBIENTAL',
        'LICENCIATURA EN EDUCACION RELIGIOSA', 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN INFORMATICA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, ESPAÑOL Y LENGUAS EXTRANJERAS',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION ARTISTICA', 'LICENCIATURA EN EDUCACIÓN ESPECIAL',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES,LENGUA CASTELLANA E INGLES',
        'LICENCIATURA EN EDUCACION BASICA  CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN LENGUA CASTELLANA E INGLES', 'LICENCIATURA EN EDUCACION INDUSTRIAL',
        'LICENCIATURA EN INGLÉS-ESPAÑOL', 'LICENCIATURA EN PEDAGOGÍA SOCIAL PARA LA REHABILITACIÓN',
        'LICENCIATURA  PEDAGOGIA INFANTIL', 'LICENCIATURA EN PEDAGOG¿A INFANTIL', 'LICENCIATURA EN PEDAGOGIA DE LA PRIMERA INFANCIA',
        'LICENCIATURA EN PREESCOLAR', 'LICENCIATURA EN INFORMATICA', 'LICENCIATURA EN TECNOLOGIA E INFORMATICA','LICENCIATURA EN INFORMATICA EDUCATIVA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN TECNOLOGIA E INFORMATICA',
        'LICENCIATURA EN EDUCACION BILINGUE', 'LICENCIATURA EN EDUCACION BASICA ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN EDUCACION BASICA, ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN EDUCACION BASICA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN LITERATURA Y LENGUA CASTELLANA', 'LICENCIATURA EN FILOSOFIA E HISTORIA',
        'LICENCIATURA EN ELECTRONICA','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: ESPAÑOL E INGLES',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES LENGUA CASTELLANA',
        'LICENCIATURA EN PRODUCCION AGROPECUARIA','LICENCIATURA EN RECREACION',
        'LICENCIATURA EN CIENCIAS SOCIALES CON ÉNFASIS EN EDUCACIÓN BÁSICA',
        'LICENCIATURA EN FILOSOFIA, PENSAMIENTO POLITICO Y ECONOMICO',
        'LICENCIATURA EN EDUCACI¿N B¿SICA PRIMARIA', 'LICENCIATURA EN FILOSOFIA Y LETRAS',
        'LICENCIATURA EN  LITERATURA Y LENGUA CASTELLANA', 'LICENCIATURA EN EDUCACIÓN PREESCOLAR',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN  HUMANIDADES, LENGUA CASTELLANA E INGLES',
        'LICENCIATURA EN EDUCACIÓN PARA LA CONVIVENCIA Y EL DESARROLLO HUMANO',
        'LICENCIATURA EN PSICOPEDAGOGIA CON ENFASIS EN ASESORIA EDUCATIVA','LICENCIATURA EN BIOLOGÍA',
        'LICENCIATURA EN EDUCACI¿N CON ¿NFASIS EN CIENCIAS SOCIALES Y AMBIENTALES',
        'LICENCIATURA EN MATEMATICAS E INFORMATICA','LICENCIATURA EN EDUCACIÓN ARTÍSTICA Y CULTURAL',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES,LENGUA CASTELLANA',
        'LICENCIATURA EN  FILOSOFIA Y CIENCIAS RELIGIOSAS', 'LICENCIATURA EN EDUCACI¿N INFANTIL',
        'LICENCIATURA EN EDUCACION BASICA PRIMARIA','LICENCIATURA EN EDUCACIÓN  FÍSICA, RECREACIÓN Y DEPORTE',
        'LICENCIATURA EN EDUCACION PREESCOLAR CON ENFASIS EN INGLES',
        'LICENCIATURA EN PEDAGOGÍA DE LA MADRE TIERRA', 'LICENCIATURA EN  FILOSOFIA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICAS E INFORMATICA',
        'LICENCIATURA EN EDUCACION RELIGIOSA CON ENFASIS EN BIBLIA',
        'LICENCIATURA EN FILOSOFIA Y HUMANIDADES','PEDAGOGÍA',
        'LICENCIATURA EN FILOSOFIA, ETICA Y VALORES HUMANOS',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN EDUCACIÓN ARTÍSTICA Y CULTURAL',
        'LICENCIATURA EN FILOSOF¿A Y HUMANIDADES','LICENCIATURA EN HUMANIDADES',
        'LICENCIATURA EN CIENCIAS NATURALES: FÍSICA QUÍMICA Y BIOLOGÍA',
        'LICENCIATURA EN PEDAGOGIA REEDUCATIVA','LICENCIATURA EN EDUCACION INICIAL',
        'LICENCIATURA  EN  CIENCIAS  SOCIALES', 'LICENCIATURA EN TECNOLOGÍA E INFORMÁTICA',
        'LICENCIATURA EN QUIMICA Y EDUCACION AMBIENTAL', 'LICENCIATURA EN EDUCACION ENFASIS EN INFORMATICA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EL AREA DE TECNOLOGIA E INFORMATICA',
        'LICENCIATURA EN EDUCACION ENFASIS EN CIENCIAS SOCIALES',
        'LICENCIATURA EN EDUCACION BASICA PRIMARIA:ENFASIS EN CIENCIAS SOCIALES',
        'LICENCIATURA EN CIENCIAS DE LA EDUCACION',
        ],

    'CIENCIAS_EXACTAS_NATURALES': [
        'LICENCIATURA EN BIOLOGIA', 'LICENCIATURA EN CIENCIAS NATURALES', 'GEOGRAFIA', 'LICENCIATURA EN QUIMICA',
        'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL', 'ZOOTECNIA', 'ADMINISTRACION AMBIENTAL',
        'MEDICINA VETERINARIA Y ZOOTECNIA',  'QUIMICA FARMACEUTICA',  'ADMINISTRACION DEL MEDIO AMBIENTE',
        'LICENCIATURA EN BIOLOGIA Y QUIMICA', 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES',
        'ADMINISTRACION MARITIMA Y FLUVIAL', 'LICENCIATURA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'BIOLOGIA', 'BACTERIOLOGIA', 'AGRONOMIA', 'BACTERIOLOGIA Y LABORATORIO CLINICO',
        'LICENCIATURA EN  CIENCIAS NATURALES Y EDUCACION AMBIENTAL',  'ADMINISTRACION Y GESTION AMBIENTAL',
        'GEOLOGIA', 'INGENIERÍA AMBIENTAL Y SANITARIA', 'INGENIERIA AMBIENTAL Y DE SANEAMIENTO', 'QUIMICA',
        'MICROBIOLOGIA Y BIOANALISIS', 'ECOLOGIA DE ZONAS COSTERAS', 'LICENCIATURA EN CIENCIAS NATURALES',
        'BIOLOGÍA', 'MICROBIOLOGIA', 'LICENCIATURA EN BIOLOGIA CON ENFASIS EN EDUCACION AMBIENTAL',
        'BIOLOGIA APLICADA', 'MICROBIOLOGIA INDUSTRIAL Y AMBIENTAL', 'GEOCIENCIAS', 'GEOLOG¿A', 'QUIMICA INDUSTRIAL',
        'ECOLOGIA', 'LICENCIATURA EN CIENCIAS NATURALES Y EDUCACIÓN AMBIENTAL',
        'LICENCIATURA EN MATEMÁTICAS Y FÍSICA', 'MICROBIOLOGÍA', 'QUIMICA.',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN CIENCIAS NATURALES Y EDUCACIÓN AMBIENTAL',
        'AGRONOMIA DEL TROPICO HUMEDO','BIOLOGIA MARINA','ESTADÍSTICA','ASTRONOMIA',
        'OCEANOGRAFIA FISICA', 'BIOLOGÍA AMBIENTAL', 'QUÍMICA INDUSTRIAL', 'QUIMICA AMBIENTAL',
        'MICROBIOLOGIA INDUSTRIAL','MICROBIOLOGÍA INDUSTRIAL Y AMBIENTAL', 'BIOQUIMICA',
        'QU¿MICA FARMAC¿UTICA','ECOLOGÍA','BIOLOGIA AMBIENTAL','MICROBIOLOGÍA Y BIOANÁLISIS',
        'GEOLOGÍA','QUÍMICA FARMACÉUTICA', 'QUÍMICA', 'OCEANOGRAFÍA', 'CIENCIAS AMBIENTALES',
        'PROFESIONAL EN AGROINDUSTRIA', 'QUIMICA Y FARMACIA','PROFESIONAL EN  AGROINDUSTRIA',
        'ESTADISTICA', 'LICENCIATURA EN MATEMATICAS', 'FÍSICA', 'FISICA', 'LICENCIATURA EN MATEMATICAS Y FISICA',
        'LICENCIATURA EN MATEMATICAS Y ESTADISTICA', 'MATEMATICAS', 'LICENCIATURA  EN MATEMATICAS', 'MATEMÁTICA APLICADA',
        'LICENCIATURA EN MATEMÁTICAS', 'LICENCIATURA EN INGLES - ESPAÑOL','MATEMATICAS CON ENFASIS EN ESTADISTICA',
        'LICENCIATURA EN MATEMATICAS Y TECNOLOGIAS DE LA INFORMACION',
        'MATEMÁTICAS', 'LICENCIATURA EN  MATEMATICAS Y FISICA', 'MATEMÁTICAS APLICADAS Y CIENCIAS DE LA COMPUTACIÓN',
        'CIENCIAS DE LA COMPUTACION','MATEMATICAS APLICADAS', 'COMPUTACIÓN CIENTÍFICA','LICENCIATURA EN MATEM¿TICAS',
        'LICENCIATURA EN FÍSICA', 'CIENCIAS DE LA COMPUTACIÓN E INTELIGENCIA ARTIFICIAL',
        ],

    'EDUCACION_FISICA': [
        'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTE','LICENCIATURA EN EDUCACION FISICA Y DEPORTES',
        'LICENCIATURA EN EDUCACIÓN FÍSICA, RECREACIÓN Y DEPORTE', 'PROFESIONAL EN DEPORTE', 'ENTRENAMIENTO DEPORTIVO',
        'CIENCIAS DEL DEPORTE Y DE LA EDUCACION FISICA', 'LICENCIATURA EN EDUCACION FISICA, DEPORTES Y RECREACION',
        'LICENCIATURA EN CIENCIAS DEL DEPORTE', 'LICENCIATURA EN EDUCACION FISICA Y DEPORTES', 'LICENCIATURA EN EDUCACION FISICA',
        'LICENCIATURA EN EDUCACIÓN FÍSICA Y DEPORTES', 'LICENCIATURA EN EDUCACION  FISICA RECREACION  Y  DEPORTE',
        'LICENCIATURA EN EDUCACION BASICA ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTE',
        'LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTES', 'LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTE',
        'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES', 'LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTE',
        'ADMINISTRACION DEPORTIVA', 'PROFESIONAL EN ACTIVIDAD FISICA Y DEPORTE', 'DEPORTE Y ACTIVIDAD FISICA',
        'CULTURA FISICA, DEPORTE Y RECREACION', 'LICENCIATURA EN CULTURA, FISICA, RECREACION Y DEPORTES',
        'LICENCIATURA EN FISICA', 'CULTURA FISICA Y DEPORTE', 'PROFESIONAL EN ENTRENAMIENTO DEPORTIVO',
        'LICENCIATURA EN DEPORTE', 'CIENCIAS DEL DEPORTE', 'DEPORTE',
        'CIENCIAS DEL DEPORTE Y  LA RECREACION', 'LICENCIATURA EN CIENCIAS DEL DEPORTE Y LA EDUCACION FISICA',
        'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTE','DEPORTE Y CULTURA FISICA',
        'ACTIVIDAD FISICA Y DEPORTE','LICENCIATURA EN EDUCACIÓN FÍSICA Y DEPORTE',
        'PROFESIONAL EN ACTIVIDAD FÍSICA Y DEPORTE','CIENCIAS DEL DEPORTE Y LA ACTIVIDAD FISICA',
        'LICENCIATURA EN EDUCACIÓN  FÍSICA, RECREACIÓN Y DEPORTES', 'GESTI¿N DEPORTIVA',
        'LICENCIATURA EN EDUCACIÓN FÍSICA DEPORTES Y RECREACIÓN', 'RECREACION',
        'PROFESIONAL EN CIENCIAS DEL DEPORTE Y LA EDUCACION FISICA',
        'FORMACION PROFESIONAL EN DEPORTE', 'DEPORTE Y ACTIVIDADA FISICA',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION  FISICA, DEPORTES Y RECREACION',
        'LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES',
        'LICENCIATURA EN EDUCACION FISICA, DEPORTE Y RECREACION ENFASIS RURAL',
        'LICENCIATURA EN EDUCACON FISICA, RECREACION Y DEPORTES',
        ],

    'IDIOMAS_LETRAS': [
        'LICENCIATURA EN LENGUAS EXTRANJERAS', 'LICENCIATURA EN INGLES', 'LITERATURA',
        'LICENCIATURA EN BILINGÜISMO CON ÉNFASIS EN INGLÉS', 'LICENCIATURA EN INGLÉS',
        'LICENCIATURA EN LENGUAS EXTRANJERAS', 'LICENCIATURA EN BILINGÜISMO CON ÉNFASIS EN INGLÉS',
        'LICENCIATURA EN LENGUA CASTELLANA,INGLES Y FRANCES', 'LICENCIATURA EN IDIOMAS EXTRANJEROS',
        'ESTUDIOS LITERARIOS', 'LICENCIATURA EN LENGUAS MODERNAS CON ENFASIS EN INGLES Y FRANCES',
        'LICENCIATURA EN INGLES', 'FILOLOGIA E IDIOMAS', 'LICENCIATURA EN LENGUA CASTELLANA E INGLES',
        'TRADUCCION INGLES-FRANCES-ESPAÑOL', 'LICENCIATURA EN LENGUA CASTELLANA Y COMUNICACION',
        'PROFESIONAL EN LENGUAS MODERNAS', 'LINGUISTICA Y LITERATURA', 'LICENCIATURA EN ESPAÑOL Y LITERATURA',
        'LINGUISTICA', 'LICENCIATURA EN LENGUAS MODERNAS', 'LENGUAS MODERNAS', 'LICENCIATURA EN ESPAÑOL Y LENGUAS EXTRANJERAS',
        'LICENCIATURA EN LENGUAS MODERNAS INGLES-FRANCES', 'LICENCIATURA EN LENGUA EXTRANJERA INGLES',
        'LETRAS - FILOLOGIA HISPANICA', 'LICENCIATURA EN IDIOMAS MODERNOS ESPAÑOL- INGLES', 'ESPAÑOL Y FILOLOGIA CLASICA',
        'LICENCIATURA EN HUMANIDADES, LENGUA CASTELLANA E INGLES', 'LICENCIATURA EN LENGUAS EXTRANJERAS: INGLES-FRANCES',
        'LICENCIATURA EN LENGUAS EXTRANJERAS INGLES-FRANCES', 'LICENCIATURA EN LENGUAS EXTRANJERAS CON ÉNFASIS EN INGLÉS',
        'LICENCIATURA EN INGLÉS Y BILINGÜISMO', 'LICENCIATURA EN LENGUAS EXTRANJERAS INGLES - FRANCES',
        'LICENCIATURA EN LENGUA CASTELLANA Y LITERATURA', 'LICENCIATURA EN LENGUA CASTELLANA',
        'LICENCIATURA EN HUMANIDADES Y LENGUA CASTELLANA', 'LICENCIATURA EN LENGUAS  EXTRANJERAS',
        'LICENCIATURA EN INGLÉS COMO LENGUA EXTRANJERA', 'LICENCIATURA EN FILOSOFIA Y LENGUA CASTELLANA',
        'LICENCIATURA EN LINGUISTICA Y LITERATURA', 'PROGRAMA PROFESIONAL UNIVERSITARIO EN LENGUAS EXTRANJERAS',
        'LICENCIATURA EN LENGUAS EXTRANJERAS CON ENFASIS EN INGLES','LENGUAS Y CULTURA', 'LICENCIATURA EN LITERATURA',
        'LICENCIATURA EN LENGUA INGLESA','LICENCIATURA EN EDUCACION CON ENFASIS EN INGLES',
        'LICENCIATURA EN EDUCACION BILINGUE CON ENFASIS EN LA ENSEÑANZA DEL INGLES',
        'LICENCIATURA EN EDUCACIÓN BILINGUE','Licenciatura en Lenguas Extranjeras con Énfasis en Inglés',
        'LICENCIATURA EN IDIOMA EXTRANJERO - INGLÉS', 'FILOLOGIA HISPANICA',
        'LICENCIATURA EN INGLES Y FRANCES','LICENCIATURA EN ESPAÑOL E INGLES',
        'LICENCIATURA EN BILINGÜISMO CON ENFASIS EN LA ENSEÑANZA DEL INGLÉS', 'CREACION LITERARIA',
        'LICENCIATURA EN EDUCACION BILINGUE-ESPAÑOL-INGLES','LICENCIATURA EN LENGUAS EXTRANJERAS CON ¿NFASIS EN INGL¿S',
        'LICENCIATURA EN BILINGUISMO CON ENFASIS EN ESPAÑOL E INGLES','LICENCIATURA  EN INGLES Y FRANCES',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUAS','LICENCIATURA EN LENGUAS MODERNAS, INGLES Y FRANCES',
        'LICENCIATURA EN LENGUAS MODERNAS CON ÉNFASIS EN INGLÉS',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN HUMANIDADES: ESPAÑOL E INGLÉS',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E IDIOMA EXTRANJERO: INGLES',
        'LICENCIATURA LENGUAS EXTRANJERAS CON ENFASIS EN INGLES','LICENCIATURA EN EDUCACIÓN BILINGÜE- ESPAÑOL E INGLES',
        'LICENCIATURA EN ESPAÑOL E INGLÉS',  'ESTUDIOS LITERARIOS Y EDICION',
        'LICENCIATURA EN ESPAÑOL Y FILOLOGÍA','LENGUAS MODERNAS Y CULTURA',
        'LICENCIATURA EN EDUCACIÓN BÁSICA PRIMARIA CON ÉNFASIS EN INGLÉS Y EN FRANCÉS',
        ],

    'ADMINISTRACION': [
        'ADMINISTRACIÓN EN SALUD OCUPACIONAL', 'ADMINISTRACION DE EMPRESAS', 'ADMINISTRACIÓN FINANCIERA',
        'ADMINISTRACION FINANCIERA', 'HOTELERIA Y TURISMO', 'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL',
        'ADMINISTRACION EN SALUD', 'ADMINISTRACIÓN DE EMPRESAS', 'SEGURIDAD Y SALUD EN EL TRABAJO',
        'ADMINISTRACION DE LA SEGURIDAD SOCIAL', 'ADMINISTRACION Y DIRECCION DE EMPRESAS', 'LOGISTICA EMPRESARIAL',
        'ADMINISTRACIÓN EN SEGURIDAD Y SALUD EN EL TRABAJO',  'ADMINISTRACION DE EMPRESAS TURISTICAS',
        'ADMINISTRACION DE NEGOCIOS INTERNACIONALES', 'ADMINISTRACION DE LA CALIDAD', 'GERENCIA EN SISTEMAS DE INFORMACION EN SALUD',
        'GESTIÓN EN ECOLOGÍA Y TURISMO', 'ADMINISTRACION PUBLICA TERRITORIAL', 'ADMINISTRACION & SERVICIO',
        'ADMINISTRACIÓN INTEGRAL DE RIESGOS DE SEGURIDAD Y SALUD EN EL TRABAJO', 'ADMINISTRACION DE LA SEGURIDAD Y SALUD OCUPACIONAL',
        'PROFESIONAL EN ADMINISTRACIÓN DE EMPRESAS', 'ADMINISTRACION DE NEGOCIOS', 'ADMINISTRACION PUBLICA',
        'ADMINISTRACION DE EMPRESAS COMERCIALES', 'ADMINISTRACION DE SISTEMAS INFORMATICOS', 'ADMINISTRACION TURISTICA Y HOTELERA',
        'ADMINISTRACION DE MERCADEO', 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS', 'ADMINISTRACION INDUSTRIAL',
        'ADMINISTRACIÓN TECNÓLOGICA', 'ADMINISTRACION', 'ADMINISTRACIÓN EN SALUD', 'ADMINISTRACION COMERCIAL Y DE MERCADEO',
        'ADMINISTRACION POLICIAL', 'ADMINISTRACION DE SERVICIOS DE SALUD', 'ADMINISTRACION EN FINANZAS Y NEGOCIOS INTERNACIONALES',
        'ADMINISTRACIÓN COMERCIAL', 'ADMINISTRACIÓN DE NEGOCIOS INTERNACIONALES', 'ADMINISTRACION HUMANA',
        'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL', 'ADMINISTRACIÓN DE EMPRESAS TURÍSTICAS Y HOTELERAS',
        'ADMINISTRACIÓN DE EMPRESAS TURÍSTICAS', 'ADMINISTRACI¿N LOG¿STICA', 'ADMINISTRACION DE SISTEMAS INFORMÁTICOS',
        'ADMINISTRACION EN SALUD: ENFASIS EN GESTION DE SERVICIOS DE SALUD Y  ENFASIS EN GESTION SANITARIA Y AMBIENTAL',
        'ADMINISTRACIÓN PÚBLICA', 'ADMINISTRACION AGROPECUARIA', 'ADMINISTRACIÓN LOGISTICA', 'ADMINISTRACIÓN DE SERVICIOS DE SALUD',
        'ADMINISTRACION AERONAUTICA', 'ADMINISTRACIÓN DE COMERCIO EXTERIOR', 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS',
        'ADMINISTRACIÓN DE LA SALUD OCUPACIONAL', 'ADMINISTRACIÓN DE EMPRESAS TURISTICAS',
        'ADMINISTRACION EN MERCADEO Y LOGISTICA INTERNACIONALES', 'ADMINISTRACION Y FINANZAS', 'ADMINISTRACIÓN TURÍSTICA',
        'ADMINISTRACION EN LOGISTICA Y PRODUCCION', 'ADMINISTRACIÓN COMERCIAL Y FINANCIERA', 'ADMINISTRACIÓN TURÍSTICA Y HOTELERA',
        'ADMINISTRACIÓN LOGÍSTICA', 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS', 'ADMINISTRACION DE INSTITUCIONES DE SERVICIO',
        'GESTION EMPRESARIAL', 'TURISMO', 'HOTELERIA Y TURISMO ECOLOGICO',
        'ADMINISTRACIÓN DE EMPRESAS - CICLO PROFESIONAL',  'ADMINISTRACIÓN DE LA SEGURIDAD Y SALUD   EN EL TRABAJO',
        'ADMINISTRACI¿N DE EMPRESAS', 'ADMINISTRACIÓN DE AGRONEGOCIOS',
        'ADMINISTRACIÒN DE NEGOCIOS INTERNACIONALES','ADMINSITRACIÓN DE EMPRESAS',
        'ADMINISTRACION DE HOTELERIA Y TURISMO','ADMINISTRACION DE EMPRESAS Y FINANZAS',
        'ADMINISTRACION LOGISTICA','ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES',
        'ADMINISTRACION COMERCIAL','ADMINISTRACION Y NEGOCIOS INTERNACIONALES',
        'PROFESIONAL SEGURIDAD Y SALUD EN EL TRABAJO','ADMINISTRACION DE GESTISN HUMANA',
        'ADMINISTRACION FINANCIERA Y DE SISTEMAS', 'ADMINISTRACION DEL TURISMO SOSTENIBLE',
        'ADMINISTRACION INFORMÁTICA','ADMINISTRACION DEL COMERCIO INTERNACIONAL','ADMINISTRACIÓN HOTELERA Y DE SERVICIOS',
        'ADMINISTRACION TURÍSTICA', 'ADMINISTRACION  FINANCIERA', 'ADMINISTRACION MARITIMA Y PORTUARIA',
        'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES','ADMINISTRACION EMPRESAS AGROINDUSTRIALES',
        'ADMINSITRACION TURISTICA Y HOTELERA','ADMINISTRACION DE EMPRESAS TURISTICA',
        'ADMINISTRACI¿N P¿BLICA','ADMINISTRACIÓN TÚRÍSTICA', 'ADMINISTRACIÓN DE EMPRESAS AGROPECUARIAS',
        'ADMINISTRACION MARITIMA','ADMINISTRACIÓN Y NEGOCIOS INTERNACIONALES',
        'ADMINISTRACIÓN DE EMPRESAS AGROINDUSTRIALES','ADMINISTRACION DE MERCADEO Y LOGISTICA INTERNACIONALES',
        'ADMINISTRACIÓN DE OBRAS DE ARQUITECTURA','ADMINISTRACION DE SISTEMAS',
        'ADMINISTRACIÓN EN SERVICIOS DE SALUD', 'ADMINISTRACIÓN AGROINDUSTRIAL',
        'ADMINISTRACIÓN MARITIMA Y PORTUARIA','ADMINISTRACION INFORMATICA',
        'ADMINISTRACION EN NEGOCIOS INTERNACIONALES', 'ADMINISTRACION DE COMERCIO EXTERIOR', 'ADMINISTRACION TURISTICA',
        'ADMINISTRACION TECNOLOGICA','SISTEMAS DE INFORMACIÓN, BIBLIOTECOLOGÍA Y ARCHIVÍSTICA',
        'ADMINISTRACIÓN DE EMPRESAS TURISTICAS POR CICLOS - CICLO PROFESIONAL',
        ],

    'MEDICINA': [
        'MEDICINA', 'ENFERMERIA',  'ODONTOLOGIA', 'PSICOLOGIA', 'MEDICINA VETERINARIA', 'PSICOLOGÍA',
        'INSTRUMENTACION QUIRURGICA', 'FISIOTERAPIA', 'FARMACIA', 'OPTOMETRIA', 'NUTRICION Y DIETETICA',
        'TERAPIA RESPIRATORIA', 'ENFERMERÍA', 'TERAPIA OCUPACIONAL', 'INTRUMENTACION QUIRURGICA',
        'FONOAUDIOLOGIA', 'SALUD OCUPACIONAL', 'OPTOMETRÍA', 'NUTRICIÓN Y DIETÉTICA', 'PROFESIONAL EN PSICOLOGÍA',
        'ODONTOLOGÍA','TERAPIA CARDIORRESPIRATORIA','BIOTECNOLOGIA','NUTRICIÓN Y DIETETICA',
        'INSTRUMENTACION QUIRURGICA PROFESIONAL', 'GERONTOLOGIA','PSICÓLOGA',
        ],

    'ARTE': [
        'MAESTRO EN MÚSICA', 'ARTE DANZARIO',  'LICENCIATURA EN EDUCACION ARTISTICA', 'LICENCIATURA EN ARTES', 'ARTES PLASTICAS',
        'LICENCIATURA EN EDUCACION ARTISTICA CON ENFASIS EN DANZA Y TEATRO', 'LICENCIATURA EN EDUCACI¿N ART¿STICA',
        'LICENCIATURA EN EDUCACION ARTISTICA Y ARTES ESCENICAS', 'LICENCIATURA EN EDUCACION ARTISTICA Y CULTURAL',
        'LICENCIATURA EN ARTES ESCENICAS CON ENFASIS EN TEATRO', 'ARTES PLÁSTICAS Y VISUALES', 'LICENCIATURA EN ARTES PLASTICAS',
        'CULINARIA Y GASTRONOMÍA', 'LICENCIATURA EN ARTES ESCENICAS', 'LICENCIATURA EN ARTES VISUALES', 'ARTES VISUALES',
        'ARTES ESCÉNICAS', 'ARTES DE LA GRABACION Y PRODUCCION MUSICAL', 'ARTES MUSICALES', 'ARTES CULINARIAS Y GASTRONOMÍA',
        'ARTE DRAMATICO', 'LICENCIATURA EN ARTE Y FOLKLORE Y CULTURA', 'ARTES DE LA ESCENA', 'FORMACION MUSICAL',
        'LICENCIATURA EN MUSICA', 'MUSICA INSTRUMENTAL', 'MUSICA', 'ESTUDIOS MUSICALES', 'GASTRONOMIA Y COCINA PROFESIONAL',
        'GASTRONOMIA', 'DANZA','ARTES ESCENICAS', 'LICENCIATURA EN ARTES ESCÉNICAS','MÚSICA',
        'LICENCIATURA EN ARTE DRAMATICO','ARTES AUDIOVISUALES','ARTE','MAESTRO EN MUSICA',
        'CINE Y TELEVISIÓN','LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION ARTISTICA-MUSICA',
        'CINE Y AUDIOVISUALES', 'ARTES PLASTICAS Y VISUALES','REALIZACIÓN EN ANIMACIÓN',
        'LICENCIATURA EN EDUCACION: ARTES PLASTICAS','MUSICA INSTRUMENTO', 'LICENCIATURA EN EDUCACIÓN ARTÍSTICA',
        'LICENCIATURA EN TEATRO', 'LICENCIATURA EN EDUCACION BASICA EN DANZA',
        'LICENCIATURA EN DANZA CLASICA','LICENCIATURA M¿SICA','LICENCIATURA EN MUSICA Y DANZA',
        'LICENCIATURA EN ARTES PLASTICAS Y VISUALES','MUSICA- CANTO',
        'LICENCIATURA EN EDUCACIÓN BÁSICA CON ÉNFASIS EN EDUCACIÓN ARTÍSTICA Y CULTURAL: MÚSICA',
        'ESTUDIOS DE ARTES LIBERALES EN CIENCIAS SOCIALES','FOTOGRAFÍA',
        'LICENCIATURA EN EDUCACIÓN: ARTES PLÁSTICAS','INTERPRETACION MUSICAL',
        'ARTES PLASTICAS CON ENFASIS EN PROCESOS BIDIMENSIONALES Y TRIDIMENSIONALES',
        'HISTORIA DEL ARTE','DANZA Y DIRECCIÓN COREOGRÁFICA','DIRECCION DE BANDA', 'TEATRO MUSICAL',
        'ARTES PLÁSTICAS','ANIMACIÓN', 'LICENCIATURA EN MÚSICA','LICENCIATURA EN ARTE Y FOLKLORE',
        'PROFESIONAL EN ARTES GASTRONOMICAS',  'PROFESIONAL EN TURISMO Y GASTRONOMÍA','GASTRONOMÍA',
        'PROFESIONAL  EN GASTRONOMÍA Y  ALTA COCINA','PROFESIONAL EN GASTRONOM¿A','PROFESIONAL EN GASTRONOMÍA Y CULINARIA',
        'PROFESIONAL EN GASTRONOMIA','PROFESIONAL EN GASTRONOMÍA','BELLAS ARTES',
        'LICENCIATURA EN ARTES ESC¿NICAS',
        ],

    'COMUNICACIONES': [
        'COMUNICACION AUDIOVISUAL', 'COMUNICACION SOCIAL  - PERIODISMO', 'COMUNICACION SOCIAL', 'INGENIERIA DE TELECOMUNICACIONES',
        'COMUNICACION Y LENGUAJES AUDIOVISUALES', 'COMUNICACION SOCIAL-PERIODISMO', 'COMUNICACION SOCIAL- PERIODISMO',
        'PROFESIONAL EN CINE Y TELEVISION', 'COMUNICACIÓN SOCIAL', 'COMUNICACION SOCIAL Y PERIODISMO',
        'CIENCIAS DE LA INFORMACION Y LA DOCUMENTACION', 'COMUNICACION SOCIAL - PERIODISMO', 'COMUNICACION AUDIOVISUAL Y MULTIMEDIOS',
        'PERIODISMO', 'LICENCIATURA EN COMUNICACION E INFORMATICA EDUCATIVA', 'PUBLICIDAD', 'CINE', 'COMUNICACION SOCIAL -  PERIODISMO',
        'LICENCIATURA EN INFORMATICA Y MEDIOS AUDIOVISUALES', 'CINE Y COMUNICACIÓN DIGITAL', 'CINE Y TELEVISION',
        'DIRECCION Y PRODUCCION DE CINE Y TELEVISION', 'COMUNICACION SOCIALY PERIODISMO', 'COMUNICACION GRAFICA PUBLICITARIA',
        'COMUNICACIÓN Y ENTRETENIMIENTO DIGITAL', 'COMUNICACION Y PERIODISMO', 'COMUNICACION AUDIOVISUAL Y MULTIMEDIAL',
        'COMUNICACION PUBLICITARIA', 'COMUNICACION SOCIAL     Y     PERIODISMO', 'FOTOGRAFÍA Y COMUNICACIÓN VISUAL',
        'PROFESIONAL EN MERCADEO Y PUBLICIDAD', 'SISTEMAS DE INFORMACION Y DOCUMENTACION', 'DIRECCION Y PRODUCCION DE MEDIOS AUDIOVISUALES',
        'PUBLICIDAD Y MERCADEO', 'PROFESIONAL UNIVERSITARIO EN FOTOGRAFÍA PARA MEDIOS', 'DIRECCION Y PRODUCCION DE RADIO Y TELEVISION',
        'MERCADEO PUBLICIDAD Y VENTAS', 'PROFESIONAL EN MEDIOS AUDIOVISUALES', 'PUBLICIDAD Y MARKETING CREATIVO',
        'COMUNICACION VISUAL','COMUNICACIÓN AUDIOVISUAL','COMUNICACION Y RELACIONES CORPORATIVAS',
        'COMUNICACION SOCIAL PERIODISMO','COMUNICACI¿N SOCIAL Y PERIODISMO',
        'PUBLICIDAD Y MARKETING DIGITAL','COMUNICACIÓN SOCIAL - PERIODISMO', 'COMUNICACION',
        'COMUNICACIÓN PUBLICITARIA','COMUNICACIONES','COMUNICACI¿N SOCIAL','MARKETING',
        'COMUNICACIÓN SOCIAL Y MEDIOS DIGITALES','COMUNICACIÓN CORPORATIVA Y RELACIONES PÚBLICAS',
        'PROFESIONAL EN COMUNICACION SOCIAL - PERIODISMO','COMUNICACION ORGANIZACIONAL',
        'COMUNICACIÓN AUDIOVISUAL Y DIGITAL','PROFESIONAL EN DIRECCIÓN Y PRODUCCIÓN DE RADIO Y TELEVISIÓN',
        'NARRATIVAS DIGITALES','COMUNICACI¿N VISUAL', 'COMUNICACIÓN CORPORATIVA',
        'COMUNICACIÓN VISUAL Y MULTIMEDIA','COMUNICACIÓN SOCIAL Y PERIODISMO',
        'ARCHIVISTICA', 'HISTORIA Y ARCHIVISTICA', 'BIBLIOTECOLOGIA', 'CIENCIA DE LA INFORMACION Y BIBLIOTECOLOGIA',
        'CIENCIA DE LA INFORMACION - BIBLIOTECOLOGIA',
        ],

    'INGENIERIA': [
        'INGENIERIA MECANICA', 'INGENIERIA INDUSTRIAL', 'INGENIERIA MECATRONICA',  'INGENIERIA CIVIL',
        'INGENIERIA AGROINDUSTRIAL', 'INGENIERIA DE PROCESOS INDUSTRIALES', 'INGENIERIA DE SISTEMAS',
        'INGENIERIA ELECTRONICA', 'INGENIERÍA INDUSTRIAL', 'INGENIERIA BIOMEDICA', 'INGENIERÍA ELECTRÓNICA',
        'INGENIERIA AMBIENTAL', 'INGENIERIA  CIVIL', 'INGENIERIA DE CONTROL', 'INGENIERIA DE DISEÑO DE PRODUCTO',
        'INGENIERIA QUIMICA', 'INGENIERIA FINANCIERA', 'INGENIERIA DE ALIMENTOS', 'INGENIERIA DE SISTEMAS Y COMPUTACION',
        'INGENIERIA AGRONOMICA', 'INGENIERÍA DE TELECOMUNICACIONES',  'INGENIERIA DE PRODUCCION',
        'INGENIERIA EN TELECOMUNICACIONES', 'INGENIERÍA MULTIMEDIA', 'INGENIERIA ELECTRICA',
        'INGENIERIA ELECTRONICA Y TELECOMUNICACIONES', 'INGENIERIA AGROPECUARIA', 'INGENIERIA ADMINISTRATIVA',
        'INGENIERIA AGROFORESTAL', 'INGENIERIA DE MATERIALES', 'INGENIERIA EN MULTIMEDIA', 'INGENIERÍA AMBIENTAL',
        'INGENIERIA DE PETROLEOS', 'INGENIERIA MULTIMEDIA', 'INGENIERIA ELECTROMECANICA', 'INGENIERÍA DE SOFTWARE',
        'INGENIERIA DE SONIDO', 'INGENIERIA EN HIGIENE Y SEGURIDAD OCUPACIONAL','INGENIERIA SANITARIA Y AMBIENTAL',
        'INGENIERÍA DE SISTEMAS', 'INGENIERIA DE PRODUCTIVIDAD Y CALIDAD', 'INGENIERIA AMBIENTAL Y SANITARIA',
        'INGENIERIA EN TOPOGRAFIA', 'INGENIER¿A EN SOFTWARE', 'INGENIERIA DE MERCADOS', 'INGENIERIA PESQUERA',
        'INGENIERÍA MECATRÓNICA', 'INGENIERIA FISICA', 'INGENIERIA EN AUTOMATIZACIÓN', 'INGENIERÍA MECÁNICA',
        'INGENIERÍA EN DISEÑO INDUSTRIAL', 'INGENIERIA FORESTAL', 'INGENIERÍA CIVIL','INGENIERIA GEOLOGICA',
        'INGENIERÍA AGRONÓMICA', 'INGENIERIA EN SEGURIDAD INDUSTRIAL E HIGIENE OCUPACIONAL', 'INGENIERÍA GEOGRÁFICA Y AMBIENTAL',
        'INGENIERIA INFORMATICA', 'INGENIERIA DE MINAS Y METALURGIA', 'INGENIERIA GEOGRAFICA Y AMBIENTAL', 'INGENIERIA AGROECOLOGICA',
        'INGENIERÍA ELECTROMECÁNICA', 'INGENIERIA DE TRANSPORTE Y VIAS', 'INGENIERIA DE SISTEMAS E INFORMATICA', 'BIOINGENIERIA',
        'INGENIERIA DE SISTEMAS Y TELECOMUNICACIONES', 'INGENIERÍA EN INFORMÁTICA', 'INGENIERÍA ELÉCTRICA',
        'INGENIERIA EN HIGIENE Y SEGURIDAD INDUSTRIAL', 'INGENIERIA EN SOFTWARE', 'INGENIERIA DE MINAS', 'INGENIERIA AERONAUTICA',
        'INGENIERIA EN ENERGIA', 'INGENIERIA AGRICOLA', 'INGENIERIA EN MECATRONICA', 'INGENIERÍA EN DISEÑO DE ENTRETENIMIENTO DIGITAL',
        'INGENIERIA EN TELEMATICA', 'INGENIERIA FINANCIERA Y DE NEGOCIOS', 'INGENIERÍA EN ENERGÍA', 'INGENIERÌA INFORMÀTICA',
        'INGENIERIA DE LA CALIDAD', 'INGENIERÍA AERONÁUTICA', 'INGENIERIA DE SOFTWARE', 'INGENIERIA EN SEGURIDAD Y SALUD PARA EL TRABAJO',
        'INGENIERÍA BIOQUÍMICA', 'INGENIERÍA DE PRODUCCIÓN', 'INGENIERIA DE PROCESOS', 'INGENIERIA METALURGICA', 'INGENIERIA SANITARIA',
        'INGENIERÍA FORESTAL', 'INGENIERIA EN CONTROL',  'INGENIERÍA DE LAS ARTES DIGITALES', 'INGENIERIA BIOTECNOLOGICA',
        'INGENIERÍA MECATRÓNICA - CICLO PROFESIONAL UNIVERSITARIO', 'INGENIERIA COMERCIAL', 'INGENIERIA TOPOGRAFICA',
        'INGENIERIA EN  TELECOMUNICACIONES', 'BIOINGENIERÍA', 'INGENIERIA CATASTRAL Y GEODESIA', 'INGENIERIA MECATRONICO',
        'INGENIERIA  AGROFORESTAL', 'INGENIERÍA AGROINDUSTRIAL', 'INGENIERÍA INFORMÁTICA', 'INGENIERIA EN INSTRUMENTACION Y CONTROL',
        'INGENIERIA AGRONÓMICA','INGENIERÍA GEOLÓGICA','INGENIERÍA ELECTRÓNICA Y TELECOMUNICACIONES',
        'INGENIER¿A DE SISTEMAS','INGENIERIA DE SISTEMAS CON ENFASIS EN TELECOMUNICACIONES',
        'INGENIERIA DE SISTEMAS Y  COMPUTACION','INGENIERIA  AMBIENTAL',
        'INGENIERÍA EN PROCESOS AGROINDUSTRIALES','INGENIERIA DE TELECOMUNICACIONES E INFORMATICA',
        'INGENIERÍA QUÍMICA','INGENIERIA NAVAL','INGENIERIA BIOLOGICA', 'INGENIERÍA NAVAL',
        'INGENIERIA BIOMÉDICA', 'INGENIERIA PECUARIA','INGENIERÍA SANITARIA',
        'INGENIERÍA DE SISTEMAS Y COMPUTACIÓN', 'INGENIIERIA DE SOFTWARE',
        'INGENIERÍA EN ENERGÍAS','INGENIERIA DE SOFTWARE Y COMUNICACIONES', 'INGENIERÍA DE PETRÓLEO Y GAS',
        'INGENIERIA EN DISEÑO DE MAQUINAS Y PRODUCTOS INDUSTRIALES','INGENIERÍA DE ALIMENTOS',
        'INGENIERIA DE SISTEMAS DE INFORMACION','INGENIERIA EN INFORMATICA','INGENIERIA TELEMATICA',
        'INGENIERIA EN PRODUCCION ACUICOLA', 'INGENIERÍA EN TECNOLOGÍAS DE LA INFORMACIÓN Y LAS COMUNICACIONES',
        'INGENIERÍA EN PROCESOS INDUSTRIALES','INGENIERÍA EN AUTOMATIZACIÓN',
        'INGENIERIA DE MANTENIMIENTO','INGENIERÍA AGRONOMICA','INGENIERIA ELÉCTRICA',
        'INGENIERÍA EN SEGURIDAD Y SALUD PARA EL TRABAJO','INGENIERÍA AGROPECUARIA',
        'INGENIERIA EN ENERGIAS','INGENIERIA DE PLASTICOS','INGENIERIA TEXTIL',
        'INGENIERÍA AGRÍCOLA','INGENIER¿A EL¿CTRICA','INGENIERIA EN AUTOMATICA INDUSTRIAL',
        'INGENIERÍA DE PETRÓLEOS','INGENIERIA DE PRODUCCION AGROINDUSTRIAL', 'INGENIERIA TELEMÁTICA',
        'INGENIERÍA EN LOGISTICA Y OPERACIONES','INGENIERIA DE SISTEMAS INFORMÁTICOS',
        'INGENIERIA EN NANOTECNOLOGIA','INGENIERIA MECÁNICA','INGENIERIA MATEMATICA',
        'INGENIERÍA BIOMÉDICA', 'INGENIERÍA PETROQUÍMICA', 'INGENIERÍA URBANA',
        'INGENIER¿A INDUSTRIAL', 'INGENIERÍA OCEANOGRÁFICA','INGENIERIA EN CONTROL Y AUTOMATIZACIÓN',
        'INGENIERÍA EN TELEMÁTICA', 'INGENIERIA EN CONTROL Y AUTOMATIZACION INDUSTRIAL',
        'INGENIERIA EN DISEÑO INDUSTRIAL','INGENIERÍA DE MATERIALES','INGENIERIA MECATRÓNICA',
        'INGENIERIA EN AUTOMATIZACION INDUSTRIAL', 'INGENIERÍA DE MINAS',
        'INGENIERÍA EN SEGURIDAD Y SALUD EN EL TRABAJO','INGENIERÍA AGROECOLÓGICA',
        ],

    'ARQUITECTURA': [
        'ARQUITECTURA', 'CONSTRUCCION EN ARQUITECTURA E INGENIERIA', 'CONSTRUCCION', 'CONSTRUCCION Y GESTION EN ARQUITECTURA',
        'GESTION Y DESARROLLO URBANOS', 'CONSTRUCCIONES CIVILES', 'GEOGRAFIA APLICADA A LA ORGANIZACION DEL ESPACIO Y PLANIFICACION REGIONAL',
        'CONSERVACION Y RESTAURACION DE BIENES MUEBLES',
        ],

    'DISEÑO': [
        'DISEÑO INDUSTRIAL',  'DISEÑO GRAFICO', 'DISEÑO VISUAL', 'PROFESIONAL EN DISEÑO GRAFICO',  'DISEÑO Y PRODUCCION DE MODA',
        'DISEÑO DE MODAS', 'PROFESIONAL  EN DISEÑO GRAFICO', 'DISEÑO', 'DISEÑO GRÁFICO',  'DISEÑO DE LA COMUNICACION GRAFICA',
        'LICENCIATURA EN DISEÑO TECNOLOGICO CON ENFASIS EN SISTEMAS MECANICOS', 'DISEÑO VISUAL DIGITAL', 'DISEÑO DIGITAL Y MULTIMEDIA',
        'DISEÑO DE COMUNICACION VISUAL', 'PROFESIONAL EN DISEÑO INDUSTRIAL', 'DISEÑO DE MEDIOS  INTERACTIVOS',
        'PROFESIONAL EN DISEÑO DE INTERIORES',  'DISEÑO DE ESPACIOS', 'DISEÑO DE ESPACIOS - ESCENARIO', 'DISEÑO DE VESTUARIO',
        'DISEÑO INTERIOR', 'DISE¿¿O DE COMUNICACI¿¿N GR¿¿FICA','DISEÑO GRÀFICO','PROFESIONAL EN GESTION DEL DISEÑO','DISEÑO DE ESPACIOS Y ESCENARIOS',
        'PROFESIONAL EN DISEÑO VISUAL', 'DISEÑO INTERACTIVO','DISEÑO CROSSMEDIA',
        'DISEÑO DE MODA','DISEÑO Y GESTIÓN DE LA MODA Y EL TEXTIL', 'PROFESIONAL EN DISEÑO DE ALTA COSTURA',
        'PROFESIONAL UNIVERSITARIO DISEÑO GRÁFICO DIGITAL','DISEÑO Y GESTIÓN DE LA MODA',
         'PROFESIONAL EN DISEÑO GRÁFICO','DISE¿O Y ADMINISTRACI¿N DE NEGOCIOS DE LA MODA',
        'DISEÑO DE COMUNICACIÓN', 'DISEÑO Y GESTIÓN DEL PRODUCTO', 'PROFESIONAL EN DISEÑO DE ESPACIOS',
        'DISE¿O CROSSMEDIA', 'DISEÑO Y GESTIÓN DE ESPACIOS','DISEÑO Y REALIZACIÓN DE MEDIOS DIGITALES',
         'PROFESIONAL EN DISEÑO DE VESTUARIO','PROFESIONAL EN DISEÑO DE MODAS',
         'PROFESIONAL UNIVERSITARIO EN DISEÑO TEXTIL PARA LA INDUSTRIA', 'DISEÑO DE MODAS Y ALTA COSTURA',
         'DISEÑO GRAFICO Y MULTIMEDIAL', 'DISENO GRAFICO',
        ],

    'CIENCIAS_MILITARES':[
        'CIENCIAS MILITARES', 'CIENCIAS NAVALES PARA OFICIALES DE INFANTERIA DE MARINA', 'CIENCIAS MILITARES AERONÁUTICAS',
        'CIENCIAS NAUTICAS PARA OFICIALES MERCANTES', 'CIENCIAS NAVALES PARA OFICIALES NAVALES',
        ]
}

def map_to_group(programa):
    for group, programas in grupo_dict.items():
        if programa in programas:
            return group
    return 'OTROS'

data['ESTU_PRGM_ACADEMICO'] = data['ESTU_PRGM_ACADEMICO'].apply(map_to_group)
conteo_programas = data['ESTU_PRGM_ACADEMICO'].value_counts()
print(conteo_programas)

ESTU_PRGM_ACADEMICO
INGENIERIA                    146581
ADMINISTRACION                130963
MEDICINA                      104885
CIENCIAS_SOCIALES              84667
CIENCIAS_ECONOMICAS            84463
PEDAGOGIA                      36258
COMUNICACIONES                 24406
CIENCIAS_EXACTAS_NATURALES     23906
IDIOMAS_LETRAS                 13337
ARQUITECTURA                   12399
DISEÑO                         10129
EDUCACION_FISICA                9652
ARTE                            9078
CIENCIAS_MILITARES              1776
Name: count, dtype: int64


In [ ]:
#Agrupación de datos por Departamento
grupo_dict = {
    'CARIBE': ['ATLANTICO', 'BOLIVAR', 'CORDOBA', 'CESAR', 'LA GUAJIRA', 'MAGDALENA', 'SUCRE', 'SAN ANDRES'],
    'ANDINA': ['ANTIOQUIA', 'BOYACA', 'CUNDINAMARCA', 'RISARALDA', 'TOLIMA', 'CALDAS', 'CAUCA', 'HUILA', 'NORTE SANTANDER', 'QUINDIO', 'SANTANDER', 'BOGOTÁ'],
    'PACIFICA': ['CHOCO', 'NARIÑO', 'VALLE'],
    'ORINOQUIA': ['ARAUCA', 'CASANARE', 'META'],
    'AMAZONICA': ['AMAZONAS', 'CAQUETA', 'GUAVIARE', 'VAUPES', 'PUTUMAYO'],
}

def map_to_group(departamento):
    for group, departamentos in grupo_dict.items():
        if departamento in departamentos:
            return group
    return 'OTROS'

data['ESTU_PRGM_DEPARTAMENTO'] = data['ESTU_PRGM_DEPARTAMENTO'].apply(map_to_group)
conteo_programas = data['ESTU_PRGM_DEPARTAMENTO'].value_counts()
print(conteo_programas)

ESTU_PRGM_DEPARTAMENTO
ANDINA       511183
CARIBE       105090
PACIFICA      62331
ORINOQUIA     10351
AMAZONICA      3545
Name: count, dtype: int64


In [ ]:
#Limpiando rendimiento global
mapeo = {'bajo': 0,
         'medio-bajo': 1,
         'medio-alto': 2,
         'alto': 3}

data.RENDIMIENTO_GLOBAL = np.r_[[mapeo[i] for i in data.RENDIMIENTO_GLOBAL]]
data.RENDIMIENTO_GLOBAL.value_counts()

,count
RENDIMIENTO_GLOBAL,
3,175619
0,172987
1,172275
2,171619


In [ ]:
#Periodo se deja tal cual esta, ya es numérico
data.PERIODO.unique()

array([20212, 20203, 20195, 20183, 20194, 20213, 20184, 20202, 20196])

In [ ]:
#Limpiando programa académico
valores = data.ESTU_PRGM_ACADEMICO.values
ESTU_PRGM_ACADEMICO_vals = sorted(np.unique(valores))
ESTU_PRGM_ACADEMICO_onehot_vals = {val: np.eye(len(ESTU_PRGM_ACADEMICO_vals))[i] for i,val in enumerate(ESTU_PRGM_ACADEMICO_vals)}
ESTU_PRGM_ACADEMICO_onehot_vals
ESTU_PRGM_ACADEMICO_onehot_enc = np.r_[[ESTU_PRGM_ACADEMICO_onehot_vals[i] for i in data.ESTU_PRGM_ACADEMICO]]
ESTU_PRGM_ACADEMICO_df = pd.DataFrame(ESTU_PRGM_ACADEMICO_onehot_enc, columns=[f"ESTU_PRGM_ACADEMICO__{v}" for v in ESTU_PRGM_ACADEMICO_onehot_vals])
ESTU_PRGM_ACADEMICO_df.head(10)

,ESTU_PRGM_ACADEMICO__ADMINISTRACION,ESTU_PRGM_ACADEMICO__ARQUITECTURA,ESTU_PRGM_ACADEMICO__ARTE,ESTU_PRGM_ACADEMICO__CIENCIAS_ECONOMICAS,ESTU_PRGM_ACADEMICO__CIENCIAS_EXACTAS_NATURALES,ESTU_PRGM_ACADEMICO__CIENCIAS_MILITARES,ESTU_PRGM_ACADEMICO__CIENCIAS_SOCIALES,ESTU_PRGM_ACADEMICO__COMUNICACIONES,ESTU_PRGM_ACADEMICO__DISEÑO,ESTU_PRGM_ACADEMICO__EDUCACION_FISICA,ESTU_PRGM_ACADEMICO__IDIOMAS_LETRAS,ESTU_PRGM_ACADEMICO__INGENIERIA,ESTU_PRGM_ACADEMICO__MEDICINA,ESTU_PRGM_ACADEMICO__PEDAGOGIA
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data = pd.concat([ESTU_PRGM_ACADEMICO_df, data], axis=1).drop('ESTU_PRGM_ACADEMICO', axis=1)
print(data.shape)

(692500, 25)


In [ ]:
#Limpiando Departamento
valores = data.ESTU_PRGM_DEPARTAMENTO.values
ESTU_PRGM_DEPARTAMENTO_vals = sorted(np.unique(valores))
ESTU_PRGM_DEPARTAMENTO_onehot_vals = {val: np.eye(len(ESTU_PRGM_DEPARTAMENTO_vals))[i] for i,val in enumerate(ESTU_PRGM_DEPARTAMENTO_vals)}
ESTU_PRGM_DEPARTAMENTO_onehot_enc = np.r_[[ESTU_PRGM_DEPARTAMENTO_onehot_vals[i] for i in data.ESTU_PRGM_DEPARTAMENTO]]
ESTU_PRGM_DEPARTAMENTO_df = pd.DataFrame(ESTU_PRGM_DEPARTAMENTO_onehot_enc, columns=[f"ESTU_PRGM_DEPARTAMENTO__{v}" for v in ESTU_PRGM_DEPARTAMENTO_onehot_vals])
data = pd.concat([ESTU_PRGM_DEPARTAMENTO_df, data], axis=1).drop('ESTU_PRGM_DEPARTAMENTO', axis=1)
print(data.shape)

(692500, 29)


In [ ]:
#Limpiando valor de matricula
data.ESTU_VALORMATRICULAUNIVERSIDAD.values[data.ESTU_VALORMATRICULAUNIVERSIDAD.isna()] = 'No info'
sum(data.ESTU_VALORMATRICULAUNIVERSIDAD.isna())
mapeo = {'Más de 7 millones': 7.75,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 500 mil y menos de 1 millón': .75,
        'Menos de 500 mil': .25,
        'No pagó matrícula': 0,
        'No info': -1}

data.ESTU_VALORMATRICULAUNIVERSIDAD = np.r_[[mapeo[i] for i in data.ESTU_VALORMATRICULAUNIVERSIDAD]]
data.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,6287


In [ ]:
#Limpiando horas de trabajo
data.ESTU_HORASSEMANATRABAJA.values[data.ESTU_HORASSEMANATRABAJA.isna()] = 'No info'
sum(data.ESTU_HORASSEMANATRABAJA.isna())
mapeo = {'0': 0,
         'Menos de 10 horas': 1,
         'Entre 11 y 20 horas': 1,
         'Entre 21 y 30 horas': 1,
         'Más de 30 horas': 1,
         'No info': -1}

data.ESTU_HORASSEMANATRABAJA = np.r_[[mapeo[i] for i in data.ESTU_HORASSEMANATRABAJA]]
data.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
1,545093
0,116550
-1,30857


In [ ]:
valores_horas_trabajo = data[data['ESTU_HORASSEMANATRABAJA'] != -1]['ESTU_HORASSEMANATRABAJA'].unique()
num_valores_faltantes = (data['ESTU_HORASSEMANATRABAJA'] == -1).sum()
data.loc[data['ESTU_HORASSEMANATRABAJA'] == -1, 'ESTU_HORASSEMANATRABAJA'] = np.random.choice(valores_horas_trabajo, num_valores_faltantes)

In [ ]:
faltantes = (data['ESTU_HORASSEMANATRABAJA'] == -1).sum()
print(faltantes)

0


In [ ]:
#Limpiando estrato
data.FAMI_ESTRATOVIVIENDA.values[data.FAMI_ESTRATOVIVIENDA.isna()] = 'No info'
sum(data.FAMI_ESTRATOVIVIENDA.isna())
mapeo = {'Estrato 6': 6,
         'Estrato 5': 5,
         'Estrato 4': 4,
         'Estrato 3': 3,
         'Estrato 2': 2,
         'Estrato 1': 1,
         'Sin Estrato': 0,
         'No info': -1}

data.FAMI_ESTRATOVIVIENDA = np.r_[[mapeo[i] for i in data.FAMI_ESTRATOVIVIENDA]]
data.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
2,232671
3,210685
1,111991
4,65514
-1,32137
5,23608
6,12605
0,3289


In [ ]:
valores_horas_trabajo = data[data['FAMI_ESTRATOVIVIENDA'] != -1]['FAMI_ESTRATOVIVIENDA'].unique()
num_valores_faltantes = (data['FAMI_ESTRATOVIVIENDA'] == -1).sum()
data.loc[data['FAMI_ESTRATOVIVIENDA'] == -1, 'FAMI_ESTRATOVIVIENDA'] = np.random.choice(valores_horas_trabajo, num_valores_faltantes)
faltantes = (data['FAMI_ESTRATOVIVIENDA'] == -1).sum()
print(faltantes)

0


In [ ]:
#Limpiando tiene internet
data.FAMI_TIENEINTERNET.values[data.FAMI_TIENEINTERNET.isna()] = 'No info'
sum(data.FAMI_TIENEINTERNET.isna())
mapeo = {'Si': 1,
         'No': 0,
         'No info': -1}

data.FAMI_TIENEINTERNET = np.r_[[mapeo[i] for i in data.FAMI_TIENEINTERNET]]
data.FAMI_TIENEINTERNET.value_counts()

,count
FAMI_TIENEINTERNET,
1,592514
0,73357
-1,26629


In [ ]:
valores_horas_trabajo = data[data['FAMI_TIENEINTERNET'] != -1]['FAMI_TIENEINTERNET'].unique()
num_valores_faltantes = (data['FAMI_TIENEINTERNET'] == -1).sum()
data.loc[data['FAMI_TIENEINTERNET'] == -1, 'FAMI_TIENEINTERNET'] = np.random.choice(valores_horas_trabajo, num_valores_faltantes)
faltantes = (data['FAMI_TIENEINTERNET'] == -1).sum()
print(faltantes)

0


In [ ]:
#Limpieza estudios padre
data.FAMI_EDUCACIONPADRE.values[data.FAMI_EDUCACIONPADRE.isna()] = 'No info'
sum(data.FAMI_EDUCACIONPADRE.isna())
mapeo = {'Postgrado': 9,
         'Educación profesional completa': 8,
         'Educación profesional incompleta': 7,
         'Técnica o tecnológica completa': 6,
         'Técnica o tecnológica incompleta': 5,
         'Secundaria (Bachillerato) completa': 4,
         'Secundaria (Bachillerato) incompleta': 3,
         'Primaria completa': 2,
         'Primaria incompleta': 1,
         'Ninguno': 0,
         'No sabe': -1,
         'No Aplica': -1,
         'No info': -1}

data.FAMI_EDUCACIONPADRE = np.r_[[mapeo[i] for i in data.FAMI_EDUCACIONPADRE]]
data.FAMI_EDUCACIONPADRE.value_counts()

,count
FAMI_EDUCACIONPADRE,
4,128289
1,125675
8,83117
3,71654
6,62995
2,55958
-1,48999
9,44169
7,27084


In [ ]:
#Limpieza estudios madre
data.FAMI_EDUCACIONMADRE.values[data.FAMI_EDUCACIONMADRE.isna()] = 'No info'
sum(data.FAMI_EDUCACIONMADRE.isna())
mapeo = {'Postgrado': 9,
         'Educación profesional completa': 8,
         'Educación profesional incompleta': 7,
         'Técnica o tecnológica completa': 6,
         'Técnica o tecnológica incompleta': 5,
         'Secundaria (Bachillerato) completa': 4,
         'Secundaria (Bachillerato) incompleta': 3,
         'Primaria completa': 2,
         'Primaria incompleta': 1,
         'Ninguno': 0,
         'No sabe': -1,
         'No Aplica': -1,
         'No info': -1}

data.FAMI_EDUCACIONMADRE = np.r_[[mapeo[i] for i in data.FAMI_EDUCACIONMADRE]]
data.FAMI_EDUCACIONMADRE.value_counts()

,count
FAMI_EDUCACIONMADRE,
4,141744
1,99420
6,89542
8,85326
3,81012
2,56125
9,46246
-1,28599
5,27533


In [ ]:
#Limpieza pago propio
data.ESTU_PAGOMATRICULAPROPIO.values[data.ESTU_PAGOMATRICULAPROPIO.isna()] = 'No info'
sum(data.ESTU_PAGOMATRICULAPROPIO.isna())
mapeo = {'Si': 1,
         'No': 0,
         'No info': -1}

data.ESTU_PAGOMATRICULAPROPIO = np.r_[[mapeo[i] for i in data.ESTU_PAGOMATRICULAPROPIO]]
data.ESTU_PAGOMATRICULAPROPIO.value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
0,382201
1,303801
-1,6498


In [ ]:
mapeo = {1: 1,
         0: 0,
         -1: 0}

data.ESTU_PAGOMATRICULAPROPIO = np.r_[[mapeo[i] for i in data.ESTU_PAGOMATRICULAPROPIO]]
data.ESTU_PAGOMATRICULAPROPIO.value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
0,388699
1,303801


#Estado de los datos luego de la limpieza

In [ ]:
print(data.dtypes)

ESTU_PRGM_DEPARTAMENTO__AMAZONICA                  float64
ESTU_PRGM_DEPARTAMENTO__ANDINA                     float64
ESTU_PRGM_DEPARTAMENTO__CARIBE                     float64
ESTU_PRGM_DEPARTAMENTO__ORINOQUIA                  float64
ESTU_PRGM_DEPARTAMENTO__PACIFICA                   float64
ESTU_PRGM_ACADEMICO__ADMINISTRACION                float64
ESTU_PRGM_ACADEMICO__ARQUITECTURA                  float64
ESTU_PRGM_ACADEMICO__ARTE                          float64
ESTU_PRGM_ACADEMICO__CIENCIAS_ECONOMICAS           float64
ESTU_PRGM_ACADEMICO__CIENCIAS_EXACTAS_NATURALES    float64
ESTU_PRGM_ACADEMICO__CIENCIAS_MILITARES            float64
ESTU_PRGM_ACADEMICO__CIENCIAS_SOCIALES             float64
ESTU_PRGM_ACADEMICO__COMUNICACIONES                float64
ESTU_PRGM_ACADEMICO__DISEÑO                        float64
ESTU_PRGM_ACADEMICO__EDUCACION_FISICA              float64
ESTU_PRGM_ACADEMICO__IDIOMAS_LETRAS                float64
ESTU_PRGM_ACADEMICO__INGENIERIA                    float

In [ ]:
data.columns

Index(['ESTU_PRGM_DEPARTAMENTO__AMAZONICA', 'ESTU_PRGM_DEPARTAMENTO__ANDINA',
       'ESTU_PRGM_DEPARTAMENTO__CARIBE', 'ESTU_PRGM_DEPARTAMENTO__ORINOQUIA',
       'ESTU_PRGM_DEPARTAMENTO__PACIFICA',
       'ESTU_PRGM_ACADEMICO__ADMINISTRACION',
       'ESTU_PRGM_ACADEMICO__ARQUITECTURA', 'ESTU_PRGM_ACADEMICO__ARTE',
       'ESTU_PRGM_ACADEMICO__CIENCIAS_ECONOMICAS',
       'ESTU_PRGM_ACADEMICO__CIENCIAS_EXACTAS_NATURALES',
       'ESTU_PRGM_ACADEMICO__CIENCIAS_MILITARES',
       'ESTU_PRGM_ACADEMICO__CIENCIAS_SOCIALES',
       'ESTU_PRGM_ACADEMICO__COMUNICACIONES', 'ESTU_PRGM_ACADEMICO__DISEÑO',
       'ESTU_PRGM_ACADEMICO__EDUCACION_FISICA',
       'ESTU_PRGM_ACADEMICO__IDIOMAS_LETRAS',
       'ESTU_PRGM_ACADEMICO__INGENIERIA', 'ESTU_PRGM_ACADEMICO__MEDICINA',
       'ESTU_PRGM_ACADEMICO__PEDAGOGIA', 'ID', 'PERIODO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADR

#Empezaremos a realizar los pasos necesarios para nuestro modelo

In [ ]:
XData, yData = data[[i for i in data.columns if i!="RENDIMIENTO_GLOBAL"]].values, data.RENDIMIENTO_GLOBAL.values
XDataTrain, XDataTest, yDataTrain, yDataTest = train_test_split(XData,yData, test_size=0.3)
print (XDataTrain.shape, yDataTrain.shape, XDataTest.shape, yDataTest.shape)

(484750, 28) (484750,) (207750, 28) (207750,)


# Modelo con CatBoostClassifier

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
model = CatBoostClassifier(
    iterations=100
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

Learning rate set to 0.5
0:	learn: 1.3180979	total: 352ms	remaining: 34.9s
1:	learn: 1.2893867	total: 580ms	remaining: 28.4s
2:	learn: 1.2737741	total: 805ms	remaining: 26s
3:	learn: 1.2654174	total: 1.05s	remaining: 25.1s
4:	learn: 1.2601114	total: 1.28s	remaining: 24.3s
5:	learn: 1.2553401	total: 1.51s	remaining: 23.7s
6:	learn: 1.2524135	total: 1.75s	remaining: 23.3s
7:	learn: 1.2498878	total: 1.99s	remaining: 22.9s
8:	learn: 1.2480553	total: 2.22s	remaining: 22.5s
9:	learn: 1.2464782	total: 2.47s	remaining: 22.2s
10:	learn: 1.2453283	total: 2.69s	remaining: 21.8s
11:	learn: 1.2442690	total: 2.94s	remaining: 21.6s
12:	learn: 1.2432415	total: 3.17s	remaining: 21.3s
13:	learn: 1.2426394	total: 3.42s	remaining: 21s
14:	learn: 1.2417586	total: 3.63s	remaining: 20.6s
15:	learn: 1.2406876	total: 3.86s	remaining: 20.3s
16:	learn: 1.2403468	total: 4.08s	remaining: 19.9s
17:	learn: 1.2398360	total: 4.31s	remaining: 19.6s
18:	learn: 1.2391460	total: 4.57s	remaining: 19.5s
19:	learn: 1.2385189

In [ ]:
model = CatBoostClassifier(
    iterations=300
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

Learning rate set to 0.299997
0:	learn: 1.3393856	total: 323ms	remaining: 1m 36s
1:	learn: 1.3113343	total: 557ms	remaining: 1m 22s
2:	learn: 1.2953740	total: 799ms	remaining: 1m 19s
3:	learn: 1.2838916	total: 1.04s	remaining: 1m 17s
4:	learn: 1.2757946	total: 1.28s	remaining: 1m 15s
5:	learn: 1.2696667	total: 1.52s	remaining: 1m 14s
6:	learn: 1.2654479	total: 1.76s	remaining: 1m 13s
7:	learn: 1.2611197	total: 2.01s	remaining: 1m 13s
8:	learn: 1.2581406	total: 2.25s	remaining: 1m 12s
9:	learn: 1.2559226	total: 2.48s	remaining: 1m 11s
10:	learn: 1.2536656	total: 2.72s	remaining: 1m 11s
11:	learn: 1.2520748	total: 2.96s	remaining: 1m 11s
12:	learn: 1.2502587	total: 3.2s	remaining: 1m 10s
13:	learn: 1.2490502	total: 3.44s	remaining: 1m 10s
14:	learn: 1.2478380	total: 3.68s	remaining: 1m 9s
15:	learn: 1.2464386	total: 3.91s	remaining: 1m 9s
16:	learn: 1.2454208	total: 4.17s	remaining: 1m 9s
17:	learn: 1.2447796	total: 4.39s	remaining: 1m 8s
18:	learn: 1.2439391	total: 4.65s	remaining: 1m 8

In [ ]:
model = CatBoostClassifier(
    iterations=500
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

Learning rate set to 0.19483
0:	learn: 1.3538253	total: 244ms	remaining: 2m 1s
1:	learn: 1.3313579	total: 508ms	remaining: 2m 6s
2:	learn: 1.3153603	total: 759ms	remaining: 2m 5s
3:	learn: 1.3031248	total: 1.01s	remaining: 2m 4s
4:	learn: 1.2941404	total: 1.25s	remaining: 2m 3s
5:	learn: 1.2863646	total: 1.49s	remaining: 2m 3s
6:	learn: 1.2799804	total: 1.74s	remaining: 2m 2s
7:	learn: 1.2751340	total: 1.99s	remaining: 2m 2s
8:	learn: 1.2708678	total: 2.23s	remaining: 2m 1s
9:	learn: 1.2673523	total: 2.49s	remaining: 2m 2s
10:	learn: 1.2646411	total: 2.73s	remaining: 2m 1s
11:	learn: 1.2622802	total: 2.98s	remaining: 2m 1s
12:	learn: 1.2600060	total: 3.21s	remaining: 2m
13:	learn: 1.2576619	total: 3.47s	remaining: 2m
14:	learn: 1.2560767	total: 3.7s	remaining: 1m 59s
15:	learn: 1.2545790	total: 3.92s	remaining: 1m 58s
16:	learn: 1.2531109	total: 4.18s	remaining: 1m 58s
17:	learn: 1.2521011	total: 4.58s	remaining: 2m 2s
18:	learn: 1.2511031	total: 4.98s	remaining: 2m 6s
19:	learn: 1.250

In [ ]:
model = CatBoostClassifier(
    iterations=800
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

Learning rate set to 0.130971
0:	learn: 1.3636628	total: 253ms	remaining: 3m 22s
1:	learn: 1.3455581	total: 491ms	remaining: 3m 15s
2:	learn: 1.3319365	total: 845ms	remaining: 3m 44s
3:	learn: 1.3210949	total: 1.28s	remaining: 4m 15s
4:	learn: 1.3120810	total: 1.72s	remaining: 4m 32s
5:	learn: 1.3044005	total: 2.12s	remaining: 4m 40s
6:	learn: 1.2975883	total: 2.54s	remaining: 4m 48s
7:	learn: 1.2918768	total: 2.96s	remaining: 4m 52s
8:	learn: 1.2869191	total: 3.39s	remaining: 4m 57s
9:	learn: 1.2825784	total: 3.79s	remaining: 4m 59s
10:	learn: 1.2787985	total: 4.2s	remaining: 5m
11:	learn: 1.2756389	total: 4.67s	remaining: 5m 6s
12:	learn: 1.2727435	total: 5.06s	remaining: 5m 6s
13:	learn: 1.2699370	total: 5.31s	remaining: 4m 57s
14:	learn: 1.2675789	total: 5.54s	remaining: 4m 50s
15:	learn: 1.2655275	total: 5.78s	remaining: 4m 43s
16:	learn: 1.2637997	total: 6.03s	remaining: 4m 37s
17:	learn: 1.2621773	total: 6.3s	remaining: 4m 33s
18:	learn: 1.2606175	total: 6.54s	remaining: 4m 28s


In [ ]:
model = CatBoostClassifier(
    iterations=1000
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

Learning rate set to 0.108464
0:	learn: 1.3673194	total: 419ms	remaining: 6m 58s
1:	learn: 1.3521393	total: 837ms	remaining: 6m 57s
2:	learn: 1.3396299	total: 1.26s	remaining: 6m 57s
3:	learn: 1.3288043	total: 1.69s	remaining: 7m 1s
4:	learn: 1.3203621	total: 2.17s	remaining: 7m 12s
5:	learn: 1.3126541	total: 2.62s	remaining: 7m 14s
6:	learn: 1.3060324	total: 2.86s	remaining: 6m 46s
7:	learn: 1.3003865	total: 3.11s	remaining: 6m 25s
8:	learn: 1.2954857	total: 3.35s	remaining: 6m 9s
9:	learn: 1.2910900	total: 3.58s	remaining: 5m 54s
10:	learn: 1.2871703	total: 3.83s	remaining: 5m 44s
11:	learn: 1.2838728	total: 4.07s	remaining: 5m 35s
12:	learn: 1.2804740	total: 4.32s	remaining: 5m 27s
13:	learn: 1.2776895	total: 4.55s	remaining: 5m 20s
14:	learn: 1.2753162	total: 4.81s	remaining: 5m 15s
15:	learn: 1.2728857	total: 5.05s	remaining: 5m 10s
16:	learn: 1.2709476	total: 5.27s	remaining: 5m 5s
17:	learn: 1.2691265	total: 5.52s	remaining: 5m 1s
18:	learn: 1.2672454	total: 5.76s	remaining: 4m 

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3687197	total: 262ms	remaining: 4m 21s
1:	learn: 1.3544041	total: 494ms	remaining: 4m 6s
2:	learn: 1.3424844	total: 726ms	remaining: 4m 1s
3:	learn: 1.3320166	total: 978ms	remaining: 4m 3s
4:	learn: 1.3236090	total: 1.23s	remaining: 4m 4s
5:	learn: 1.3169287	total: 1.48s	remaining: 4m 5s
6:	learn: 1.3103915	total: 1.72s	remaining: 4m 3s
7:	learn: 1.3043261	total: 1.96s	remaining: 4m 2s
8:	learn: 1.2992524	total: 2.21s	remaining: 4m 3s
9:	learn: 1.2947563	total: 2.46s	remaining: 4m 4s
10:	learn: 1.2906327	total: 2.71s	remaining: 4m 3s
11:	learn: 1.2871196	total: 2.94s	remaining: 4m 1s
12:	learn: 1.2839946	total: 3.18s	remaining: 4m 1s
13:	learn: 1.2808115	total: 3.42s	remaining: 4m 1s
14:	learn: 1.2779673	total: 3.66s	remaining: 4m
15:	learn: 1.2756435	total: 3.9s	remaining: 3m 59s
16:	learn: 1.2735394	total: 4.14s	remaining: 3m 59s
17:	learn: 1.2714797	total: 4.4s	remaining: 3m 59s
18:	learn: 1.2697149	total: 4.64s	remaining: 3m 59s
19:	learn: 1.2677214	total: 4.88s	remaini

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.2
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3530640	total: 241ms	remaining: 4m
1:	learn: 1.3302964	total: 466ms	remaining: 3m 52s
2:	learn: 1.3141840	total: 697ms	remaining: 3m 51s
3:	learn: 1.3019000	total: 955ms	remaining: 3m 57s
4:	learn: 1.2928371	total: 1.2s	remaining: 3m 58s
5:	learn: 1.2859358	total: 1.43s	remaining: 3m 56s
6:	learn: 1.2791676	total: 1.68s	remaining: 3m 57s
7:	learn: 1.2742762	total: 1.93s	remaining: 3m 59s
8:	learn: 1.2700461	total: 2.17s	remaining: 3m 58s
9:	learn: 1.2669903	total: 2.4s	remaining: 3m 57s
10:	learn: 1.2644290	total: 2.64s	remaining: 3m 57s
11:	learn: 1.2616329	total: 2.89s	remaining: 3m 58s
12:	learn: 1.2596385	total: 3.14s	remaining: 3m 58s
13:	learn: 1.2579103	total: 3.37s	remaining: 3m 57s
14:	learn: 1.2563496	total: 3.6s	remaining: 3m 56s
15:	learn: 1.2545020	total: 3.85s	remaining: 3m 57s
16:	learn: 1.2532809	total: 4.22s	remaining: 4m 3s
17:	learn: 1.2521240	total: 4.64s	remaining: 4m 12s
18:	learn: 1.2508850	total: 5.07s	remaining: 4m 22s
19:	learn: 1.2498988	total: 5.

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=4,

)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3713895	total: 215ms	remaining: 3m 35s
1:	learn: 1.3592772	total: 410ms	remaining: 3m 24s
2:	learn: 1.3492950	total: 609ms	remaining: 3m 22s
3:	learn: 1.3406347	total: 840ms	remaining: 3m 29s
4:	learn: 1.3328207	total: 1.04s	remaining: 3m 27s
5:	learn: 1.3262616	total: 1.24s	remaining: 3m 25s
6:	learn: 1.3199262	total: 1.46s	remaining: 3m 26s
7:	learn: 1.3146781	total: 1.65s	remaining: 3m 24s
8:	learn: 1.3101228	total: 1.86s	remaining: 3m 25s
9:	learn: 1.3057637	total: 2.08s	remaining: 3m 25s
10:	learn: 1.3020182	total: 2.27s	remaining: 3m 24s
11:	learn: 1.2988274	total: 2.47s	remaining: 3m 23s
12:	learn: 1.2954932	total: 2.68s	remaining: 3m 23s
13:	learn: 1.2927208	total: 2.9s	remaining: 3m 24s
14:	learn: 1.2900942	total: 3.11s	remaining: 3m 24s
15:	learn: 1.2878181	total: 3.32s	remaining: 3m 24s
16:	learn: 1.2855732	total: 3.51s	remaining: 3m 23s
17:	learn: 1.2833575	total: 3.72s	remaining: 3m 23s
18:	learn: 1.2816201	total: 3.92s	remaining: 3m 22s
19:	learn: 1.2799910	to

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,

)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3687197	total: 241ms	remaining: 4m
1:	learn: 1.3544041	total: 479ms	remaining: 3m 58s
2:	learn: 1.3424844	total: 705ms	remaining: 3m 54s
3:	learn: 1.3320166	total: 953ms	remaining: 3m 57s
4:	learn: 1.3236090	total: 1.2s	remaining: 3m 58s
5:	learn: 1.3169287	total: 1.45s	remaining: 4m
6:	learn: 1.3103915	total: 1.69s	remaining: 3m 59s
7:	learn: 1.3043261	total: 1.96s	remaining: 4m 3s
8:	learn: 1.2992524	total: 2.35s	remaining: 4m 18s
9:	learn: 1.2947563	total: 2.77s	remaining: 4m 33s
10:	learn: 1.2906327	total: 3.15s	remaining: 4m 42s
11:	learn: 1.2871196	total: 3.56s	remaining: 4m 52s
12:	learn: 1.2839946	total: 3.94s	remaining: 4m 59s
13:	learn: 1.2808115	total: 4.35s	remaining: 5m 6s
14:	learn: 1.2779673	total: 4.76s	remaining: 5m 12s
15:	learn: 1.2756435	total: 5.17s	remaining: 5m 17s
16:	learn: 1.2735394	total: 5.58s	remaining: 5m 22s
17:	learn: 1.2714797	total: 6.08s	remaining: 5m 31s
18:	learn: 1.2697149	total: 6.56s	remaining: 5m 38s
19:	learn: 1.2677214	total: 6.83s

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=10,

)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3652490	total: 745ms	remaining: 12m 24s
1:	learn: 1.3486562	total: 1.76s	remaining: 14m 37s
2:	learn: 1.3348005	total: 2.74s	remaining: 15m 10s
3:	learn: 1.3228919	total: 3.74s	remaining: 15m 31s
4:	learn: 1.3127511	total: 4.88s	remaining: 16m 11s
5:	learn: 1.3039542	total: 5.45s	remaining: 15m 2s
6:	learn: 1.2965818	total: 6.03s	remaining: 14m 15s
7:	learn: 1.2901339	total: 6.62s	remaining: 13m 40s
8:	learn: 1.2844726	total: 7.19s	remaining: 13m 11s
9:	learn: 1.2794222	total: 7.78s	remaining: 12m 49s
10:	learn: 1.2752786	total: 8.35s	remaining: 12m 30s
11:	learn: 1.2716359	total: 8.92s	remaining: 12m 14s
12:	learn: 1.2681338	total: 9.49s	remaining: 12m
13:	learn: 1.2650663	total: 10.1s	remaining: 11m 49s
14:	learn: 1.2623455	total: 10.6s	remaining: 11m 39s
15:	learn: 1.2598832	total: 11.2s	remaining: 11m 30s
16:	learn: 1.2576731	total: 11.8s	remaining: 11m 21s
17:	learn: 1.2556945	total: 12.4s	remaining: 11m 14s
18:	learn: 1.2540472	total: 12.9s	remaining: 11m 7s
19:	learn

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    l2_leaf_reg = 1

)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3651035	total: 575ms	remaining: 9m 34s
1:	learn: 1.3484592	total: 1.15s	remaining: 9m 35s
2:	learn: 1.3345679	total: 1.72s	remaining: 9m 31s
3:	learn: 1.3226407	total: 2.31s	remaining: 9m 34s
4:	learn: 1.3124607	total: 2.88s	remaining: 9m 34s
5:	learn: 1.3036335	total: 3.45s	remaining: 9m 32s
6:	learn: 1.2962844	total: 4.04s	remaining: 9m 33s
7:	learn: 1.2900703	total: 4.61s	remaining: 9m 31s
8:	learn: 1.2843398	total: 5.19s	remaining: 9m 31s
9:	learn: 1.2793200	total: 5.77s	remaining: 9m 31s
10:	learn: 1.2749994	total: 6.35s	remaining: 9m 30s
11:	learn: 1.2713262	total: 6.92s	remaining: 9m 29s
12:	learn: 1.2678351	total: 7.5s	remaining: 9m 29s
13:	learn: 1.2648186	total: 8.07s	remaining: 9m 28s
14:	learn: 1.2621144	total: 8.88s	remaining: 9m 42s
15:	learn: 1.2596022	total: 9.9s	remaining: 10m 8s
16:	learn: 1.2573515	total: 10.9s	remaining: 10m 28s
17:	learn: 1.2552766	total: 11.9s	remaining: 10m 47s
18:	learn: 1.2534899	total: 12.9s	remaining: 11m 5s
19:	learn: 1.2517813	t

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    l2_leaf_reg = 5

)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3653776	total: 589ms	remaining: 9m 48s
1:	learn: 1.3488363	total: 1.16s	remaining: 9m 37s
2:	learn: 1.3350102	total: 1.75s	remaining: 9m 40s
3:	learn: 1.3231173	total: 2.31s	remaining: 9m 35s
4:	learn: 1.3130098	total: 2.91s	remaining: 9m 38s
5:	learn: 1.3042344	total: 3.49s	remaining: 9m 38s
6:	learn: 1.2968957	total: 4.06s	remaining: 9m 35s
7:	learn: 1.2904446	total: 4.64s	remaining: 9m 35s
8:	learn: 1.2847910	total: 5.2s	remaining: 9m 32s
9:	learn: 1.2798194	total: 5.79s	remaining: 9m 33s
10:	learn: 1.2758379	total: 6.35s	remaining: 9m 31s
11:	learn: 1.2720769	total: 6.93s	remaining: 9m 30s
12:	learn: 1.2687994	total: 7.5s	remaining: 9m 29s
13:	learn: 1.2658123	total: 8.07s	remaining: 9m 28s
14:	learn: 1.2632021	total: 8.77s	remaining: 9m 35s
15:	learn: 1.2608278	total: 9.81s	remaining: 10m 3s
16:	learn: 1.2585808	total: 10.8s	remaining: 10m 24s
17:	learn: 1.2564703	total: 11.8s	remaining: 10m 43s
18:	learn: 1.2545873	total: 12.9s	remaining: 11m 6s
19:	learn: 1.2528311	t

# Parametros seleccionados

In [ ]:
model = CatBoostClassifier(
    iterations=800,
    learning_rate=0.1,
    depth=4,
    l2_leaf_reg = 10,
    loss_function='MultiClass',
    border_count = 100
)
model.fit(XDataTrain, yDataTrain)

predictions = model.predict(XDataTest)

accuracy = accuracy_score(yDataTest, predictions)
print(f'Accuracy: {accuracy:.4f}')

0:	learn: 1.3714153	total: 377ms	remaining: 5m 1s
1:	learn: 1.3593282	total: 765ms	remaining: 5m 5s
2:	learn: 1.3493528	total: 1.15s	remaining: 5m 6s
3:	learn: 1.3406919	total: 1.46s	remaining: 4m 50s
4:	learn: 1.3328770	total: 1.67s	remaining: 4m 25s
5:	learn: 1.3263239	total: 1.86s	remaining: 4m 5s
6:	learn: 1.3199871	total: 2.08s	remaining: 3m 55s
7:	learn: 1.3147380	total: 2.29s	remaining: 3m 46s
8:	learn: 1.3101827	total: 2.48s	remaining: 3m 38s
9:	learn: 1.3058220	total: 2.69s	remaining: 3m 32s
10:	learn: 1.3020764	total: 2.9s	remaining: 3m 27s
11:	learn: 1.2988891	total: 3.12s	remaining: 3m 24s
12:	learn: 1.2955542	total: 3.31s	remaining: 3m 20s
13:	learn: 1.2927830	total: 3.52s	remaining: 3m 17s
14:	learn: 1.2901585	total: 3.72s	remaining: 3m 14s
15:	learn: 1.2878855	total: 3.92s	remaining: 3m 12s
16:	learn: 1.2856397	total: 4.13s	remaining: 3m 10s
17:	learn: 1.2834221	total: 4.34s	remaining: 3m 8s
18:	learn: 1.2816833	total: 4.52s	remaining: 3m 5s
19:	learn: 1.2800567	total: 4